<a href="https://colab.research.google.com/github/heyns1000/Muller.faa.zone/blob/main/FAA_Edge_Database_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
import datetime
import random # Move the import here

# --- Configuration ---
DB_NAME = 'faa_edge_database.db'

# --- Mock Data for Testing AI Modules ---

# 1. Sales Orders Table (For Production Planning AI)
sales_orders_data = {
    'sales_order_number': ['SO-001', 'SO-002', 'SO-003', 'SO-004', 'SO-005'],
    'profile': ['Cove', 'Biance', 'Sante', 'Lisa', 'Gloria'],
    'size_mm': [85, 140, 85, 185, 185],
    'quantity_pcs': [800, 1200, 1500, 500, 2000],
    'status': ['Pending', 'Pending', 'Pending', 'Scheduled', 'Pending'],
    'client_name': ['Client A', 'Client B', 'Client C', 'Client D', 'Client E']
}
sales_orders = pd.DataFrame(sales_orders_data)

# 2. Deliveries Table (For Route Optimization AI)
deliveries_data = {
    'sales_order_number': ['SO-001', 'SO-002', 'SO-003', 'SO-005'],
    'client_name': ['Client A', 'Client B', 'Client C', 'Client E'],
    # Mock addresses for Pretoria/JHB area for Google Maps route testing
    'address': [
        "Menlyn Park Shopping Centre, Pretoria",
        "Sandton City, Johannesburg",
        "The Grove Mall, Pretoria",
        "Mall of Africa, Midrand"
    ],
    'quantity_pcs': [800, 1200, 1500, 2000],
    'status': ['Scheduled', 'Scheduled', 'Scheduled', 'Scheduled']
}
deliveries = pd.DataFrame(deliveries_data)

# 3. Financial Transactions Table (For Cash Flow AI)
finance_data = {
    'date': [
        (datetime.datetime.now() - datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        for i in range(10, 0, -1)
    ],
    'transaction_id': [f'TX-{i}' for i in range(10)],
    'revenue': [random.randint(15000, 35000) for _ in range(10)],
    'payment_type': ['EFT', 'COD', 'EFT', 'COD', 'EFT', 'EFT', 'COD', 'EFT', 'COD', 'EFT'],
    'payment_received': [True, False, True, True, True, True, False, True, True, True],
    'client_name': [f'Client {random.choice("FGHIJ")}' for _ in range(10)],
    'sales_order_number': [f'SO-{random.randint(10, 99)}' for _ in range(10)]
}
finance_transactions = pd.DataFrame(finance_data)


# --- Database Initialization Function ---
def setup_database():
    """Initializes the SQLite database and populates mock tables."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    print(f"Creating tables in {DB_NAME}...")

    # Drop existing tables to ensure clean setup
    cursor.execute("DROP TABLE IF EXISTS sales_orders")
    cursor.execute("DROP TABLE IF EXISTS deliveries")
    cursor.execute("DROP TABLE IF EXISTS financial_transactions")

    # Write new dataframes to SQL
    sales_orders.to_sql('sales_orders', conn, if_exists='replace', index=False)
    deliveries.to_sql('deliveries', conn, if_exists='replace', index=False)
    finance_transactions.to_sql('financial_transactions', conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()
    print("Database setup complete. Tables created: sales_orders, deliveries, financial_transactions.")

if __name__ == '__main__':
    setup_database()

Creating tables in faa_edge_database.db...
Database setup complete. Tables created: sales_orders, deliveries, financial_transactions.


In [ ]:
from geopy.geocoders import Nominatim
import numpy as np

geolocator = Nominatim(user_agent="delivery_geocoder")

def geocode_address(address):
    """Geocodes an address string and returns latitude and longitude."""
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding address '{address}': {e}")
        return (None, None)

deliveries[['latitude', 'longitude']] = deliveries['address'].apply(geocode_address).apply(pd.Series)

display(deliveries)

,sales_order_number,client_name,address,quantity_pcs,status,latitude,longitude
0,SO-001,Client A,"Menlyn Park Shopping Centre, Pretoria",800,Scheduled,-25.782656,28.275067
1,SO-002,Client B,"Sandton City, Johannesburg",1200,Scheduled,-26.109043,28.052405
2,SO-003,Client C,"The Grove Mall, Pretoria",1500,Scheduled,NaN,NaN
3,SO-005,Client E,"Mall of Africa, Midrand",2000,Scheduled,-26.015085,28.107395


In [ ]:
display(delivery_map)

NameError: name 'delivery_map' is not defined

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim(user_agent="delivery_geocoder", timeout=5) # Increase timeout

address_to_geocode = deliveries[deliveries['latitude'].isna()]['address'].iloc[0]
print(f"Attempting to re-geocode: {address_to_geocode}")

try:
    location = geolocator.geocode(address_to_geocode)
    if location:
        print(f"Successfully geocoded: Latitude - {location.latitude}, Longitude - {location.longitude}")
    else:
        print("Geocoding failed for this address.")
except GeocoderTimedOut:
    print("Geocoding timed out again. The geocoding service might be slow or unresponsive.")
except Exception as e:
    print(f"An error occurred during geocoding: {e}")

Attempting to re-geocode: The Grove Mall, Pretoria
Geocoding failed for this address.


In [ ]:
import folium

# Calculate the average latitude and longitude for centering the map
valid_deliveries = deliveries.dropna(subset=['latitude', 'longitude'])
if not valid_deliveries.empty:
    avg_latitude = valid_deliveries['latitude'].mean()
    avg_longitude = valid_deliveries['longitude'].mean()
    map_center = [avg_latitude, avg_longitude]
else:
    # Default center if no valid locations
    map_center = [-25.7479, 28.2293] # Pretoria coordinates

# Create a Folium map
delivery_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each delivery location with valid coordinates
delivery_coordinates = []
for index, row in deliveries.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"<b>Client:</b> {row['client_name']}<br><b>Address:</b> {row['address']}<br><b>Quantity:</b> {row['quantity_pcs']}"
        ).add_to(delivery_map)
        delivery_coordinates.append((row['latitude'], row['longitude']))

# Add a polyline connecting the delivery locations (if there are valid locations)
if delivery_coordinates:
    folium.PolyLine(delivery_coordinates, color="blue", weight=2.5, opacity=1).add_to(delivery_map)

# Display the map
display(delivery_map)

In [ ]:
import folium

# Calculate the average latitude and longitude for centering the map
valid_deliveries = deliveries.dropna(subset=['latitude', 'longitude'])
if not valid_deliveries.empty:
    avg_latitude = valid_deliveries['latitude'].mean()
    avg_longitude = valid_deliveries['longitude'].mean()
    map_center = [avg_latitude, avg_longitude]
else:
    # Default center if no valid locations
    map_center = [-25.7479, 28.2293] # Pretoria coordinates

# Create a Folium map
delivery_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each delivery location with valid coordinates
delivery_coordinates = []
for index, row in deliveries.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"<b>Client:</b> {row['client_name']}<br><b>Address:</b> {row['address']}<br><b>Quantity:</b> {row['quantity_pcs']}"
        ).add_to(delivery_map)
        delivery_coordinates.append((row['latitude'], row['longitude']))

# Add a polyline connecting the delivery locations (if there are valid locations)
if delivery_coordinates:
    folium.PolyLine(delivery_coordinates, color="blue", weight=2.5, opacity=1).add_to(delivery_map)

# Display the map
display(delivery_map)

In [ ]:
%pip install geopy folium

# Task
Visualize delivery routes from the 'deliveries' DataFrame.

## Install necessary libraries

### Subtask:
Install libraries for geocoding and mapping, such as `geopy` and `folium`.


**Reasoning**:
The subtask requires installing the `geopy` and `folium` libraries. I will use pip to install both libraries in a single code block.



In [ ]:
%pip install geopy folium

## Geocode addresses

### Subtask:
Convert the addresses in the 'deliveries' DataFrame into latitude and longitude coordinates using a geocoding service.


**Reasoning**:
Convert the addresses in the 'deliveries' DataFrame into latitude and longitude coordinates using a geocoding service.



In [ ]:
from geopy.geocoders import Nominatim
import numpy as np

geolocator = Nominatim(user_agent="delivery_geocoder")

def geocode_address(address):
    """Geocodes an address string and returns latitude and longitude."""
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding address '{address}': {e}")
        return (None, None)

deliveries[['latitude', 'longitude']] = deliveries['address'].apply(geocode_address).apply(pd.Series)

display(deliveries)

In [ ]:
import folium

# Calculate the average latitude and longitude for centering the map
valid_deliveries = deliveries.dropna(subset=['latitude', 'longitude'])
if not valid_deliveries.empty:
    avg_latitude = valid_deliveries['latitude'].mean()
    avg_longitude = valid_deliveries['longitude'].mean()
    map_center = [avg_latitude, avg_longitude]
else:
    # Default center if no valid locations
    map_center = [-25.7479, 28.2293] # Pretoria coordinates

# Create a Folium map
delivery_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each delivery location with valid coordinates
delivery_coordinates = []
for index, row in deliveries.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"<b>Client:</b> {row['client_name']}<br><b>Address:</b> {row['address']}<br><b>Quantity:</b> {row['quantity_pcs']}"
        ).add_to(delivery_map)
        delivery_coordinates.append((row['latitude'], row['longitude']))

# Add a polyline connecting the delivery locations (if there are valid locations)
if delivery_coordinates:
    folium.PolyLine(delivery_coordinates, color="blue", weight=2.5, opacity=1).add_to(delivery_map)

# Display the map
display(delivery_map)

## Visualize routes

### Subtask:
Use a mapping library like Folium to create a map and draw lines representing the delivery routes based on the geocoded coordinates. Add markers for each delivery location.


**Reasoning**:
Create a Folium map, add markers for each delivery location with valid coordinates, extract the valid coordinates into a list, and draw a polyline connecting these points. Then display the map.



In [ ]:
import folium

# Calculate the average latitude and longitude for centering the map
valid_deliveries = deliveries.dropna(subset=['latitude', 'longitude'])
if not valid_deliveries.empty:
    avg_latitude = valid_deliveries['latitude'].mean()
    avg_longitude = valid_deliveries['longitude'].mean()
    map_center = [avg_latitude, avg_longitude]
else:
    # Default center if no valid locations
    map_center = [-25.7479, 28.2293] # Pretoria coordinates

# Create a Folium map
delivery_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each delivery location with valid coordinates
delivery_coordinates = []
for index, row in deliveries.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"<b>Client:</b> {row['client_name']}<br><b>Address:</b> {row['address']}<br><b>Quantity:</b> {row['quantity_pcs']}"
        ).add_to(delivery_map)
        delivery_coordinates.append((row['latitude'], row['longitude']))

# Add a polyline connecting the delivery locations (if there are valid locations)
if delivery_coordinates:
    folium.PolyLine(delivery_coordinates, color="blue", weight=2.5, opacity=1).add_to(delivery_map)

# Display the map
display(delivery_map)

## Display the map

### Subtask:
Display the generated map.


**Reasoning**:
Display the generated Folium map object.



In [ ]:
display(delivery_map)

## Summary:

### Data Analysis Key Findings

*   The necessary libraries for geocoding and mapping (`geopy` and `folium`) were already installed.
*   Geocoding of addresses was mostly successful, adding 'latitude' and 'longitude' columns to the DataFrame.
*   One address ('The Grove Mall, Pretoria') could not be geocoded due to `ReadTimeoutError`, resulting in missing coordinate data for that entry.
*   A Folium map was successfully generated, centered on the average coordinates of the valid delivery locations.
*   Markers were added to the map for each delivery location with valid coordinates, including popups with client and delivery information.
*   A polyline was drawn on the map connecting the valid delivery locations, visualizing a potential route.

### Insights or Next Steps

*   Investigate the geocoding failure for 'The Grove Mall, Pretoria' to ensure all locations can be visualized in the future. This might involve retrying the geocoding or using an alternative service or manual lookup.
*   Consider incorporating the order of deliveries or optimizing the route shown on the map in future visualizations, as the current polyline simply connects points in the order they appear in the DataFrame.


# Task
Develop a production planning AI module using the "sales_orders" dataset.

## Understand the production planning problem

### Subtask:
Define the specific problem the AI module will solve (e.g., optimize production schedule, predict material needs, forecast demand).


**Reasoning**:
Analyze the `sales_orders` DataFrame to understand the available data for defining the production planning problem.



In [ ]:
display(sales_orders.head())
display(sales_orders.info())
display(sales_orders.describe())

## Data preparation for production planning

### Subtask:
Identify and prepare the relevant data from the 'sales_orders' and potentially other sources. This may involve cleaning, transforming, and feature engineering the data.


**Reasoning**:
Filter the `sales_orders` DataFrame to include only rows where the 'status' column is 'Pending', select the specified columns, store the result in `pending_orders`, and then display the head and info of the new DataFrame.



In [ ]:
pending_orders = sales_orders[sales_orders['status'] == 'Pending'][['sales_order_number', 'profile', 'size_mm', 'quantity_pcs']]
display(pending_orders.head())
display(pending_orders.info())

## Choose an ai model

### Subtask:
Select an appropriate AI model or technique based on the problem definition and data characteristics (e.g., forecasting models, optimization algorithms, machine learning models).


## Develop and train the model

### Subtask:
Implement the chosen model and train it using the prepared data.


**Reasoning**:
Define a scoring function for pending orders based on quantity and apply it to the pending_orders DataFrame to generate a priority score for each order, then sort the DataFrame by this score and display the result.



In [ ]:
def calculate_priority_score(row):
    """Calculates a priority score for a pending order."""
    # Example scoring: Prioritize larger quantities
    return row['quantity_pcs']

pending_orders['priority_score'] = pending_orders.apply(calculate_priority_score, axis=1)
pending_orders_prioritized = pending_orders.sort_values(by='priority_score', ascending=False)

display(pending_orders_prioritized)

## Evaluate the model

### Subtask:
Evaluate the prioritization approach by examining the resulting prioritized list.


## Integrate and deploy (optional)

### Subtask:
Outline potential steps for integrating the production planning module into a larger system or deploying it for use.


## Summary:

### Data Analysis Key Findings

*   The available `sales_orders` dataset contains information about product profiles, sizes, quantities, order status, and clients, which are relevant for production planning.
*   The initial analysis identified that the dataset lacks explicit information about production timelines, deadlines, resource constraints, or production processes.
*   Based on the available data, the defined problem for the AI module is prioritizing and generating a basic production schedule for pending sales orders.
*   Data preparation involved filtering the `sales_orders` dataset to include only pending orders and selecting the 'sales\_order\_number', 'profile', 'size\_mm', and 'quantity\_pcs' columns into a `pending_orders` DataFrame.
*   A simple rule-based or scoring system was chosen as an appropriate AI technique for prioritization, given the data and problem scope.
*   A priority score was calculated for each pending order based solely on the order quantity, with higher quantities receiving higher scores.
*   The pending orders were successfully prioritized by sorting them in descending order based on the calculated priority score.
*   Evaluation of the prioritization approach revealed that solely prioritizing based on quantity is a very basic strategy and does not consider crucial factors for real-world production planning such as deadlines, customer priority, order value, resource availability, or production time.
*   Potential integration and deployment steps include saving the prioritized data to a database, exposing it via an API, generating reports, and considering execution frequency, computational resources, and monitoring.

### Insights or Next Steps

*   The current prioritization model is overly simplistic and needs to incorporate additional factors such as deadlines, customer value, or resource constraints to be more effective for real-world production planning.
*   Further steps should involve gathering more comprehensive data on production processes, resource availability, and order deadlines to develop a more sophisticated and practical production scheduling AI module.
